# Projeção Populacional da RMC entre 2022 e 2042

## Preparação do _Notebook_

Esse documento, por ser construído em Linguagem Python, apresenta textos e código, por essa razão, sua preparação depende de importação das bibliotecas que serão utilizadas nos procedimentos analíticos.

In [83]:
import pandas as pd # importação de biblioteca de manipulação de dados tabulares
import matplotlib.pyplot as plt # importação de biblioteca de plotagem de gráficos 
import numpy as np # importação de biblioteca para cálculos matemáticos
from scipy.optimize import curve_fit # importação de função a ser utilizada para ajuste de modelo preditivo 
from itertools import product # importação de função voltada para otimização de laços de iteração

### Carregamento dos Dados de residentes da RMC que moravam em outros estados em 2005

In [84]:
df_populacao = pd.read_csv('../source/tab/results/pop_1996_2022_estimativa_intercensitaria.csv', sep=';', decimal=',')
df_populacao = df_populacao[['Sexo', 'Faixa Etária', '2022']]
#df_populacao.set_index(['Sexo', 'Faixa Etária'], inplace=True)
df_populacao.head(2)

,Sexo,Faixa Etária,2022
0,Feminino,Menor 1 ano,22722
1,Feminino,1 a 4 anos,96868


In [85]:
df_mortalidade = pd.read_csv('../source/tab/results/mortalidade_projetada_rmc_2022_2042.csv', sep=';', decimal=',', encoding='utf-16', index_col=0)
df_mortalidade.drop(columns=['Constante', 'Coeficiente'], inplace=True)
#df_mortalidade.set_index(['Sexo', 'Faixa Etária'], inplace=True)
df_mortalidade.head(2)

,Sexo,Faixa Etária,2022,2023,2024,2025,2026,2027,2028,2029,...,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042
0,Feminino,Menor 1 ano,6.707037,6.436855,6.177556,5.928703,5.689874,5.460667,5.240692,5.029579,...,4.266813,4.094931,3.929973,3.771660,3.619725,3.473910,3.333969,3.199665,3.070771,2.947070
1,Feminino,1 a 4 anos,0.216512,0.204462,0.193083,0.182338,0.172190,0.162607,0.153558,0.145012,...,0.115327,0.108909,0.102847,0.097124,0.091719,0.086614,0.081794,0.077242,0.072943,0.068884


In [86]:
df_natalidade = pd.read_csv('../source/tab/results/natalidade_projetada_rmc_2022_2042.csv', sep=';', decimal=',', encoding='utf-16', index_col=0)

In [87]:
df_saldo_migratorio = pd.read_csv('../source/tab/results/saldo_migratorio_2010.csv', sep=';', decimal=',', encoding='utf-16')

In [88]:
df_mortalidade

,Sexo,Faixa Etária,2022,2023,2024,2025,2026,2027,2028,2029,...,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042
0,Feminino,Menor 1 ano,6.707037,6.436855,6.177556,5.928703,5.689874,5.460667,5.240692,5.029579,...,4.266813,4.094931,3.929973,3.771660,3.619725,3.473910,3.333969,3.199665,3.070771,2.947070
1,Feminino,1 a 4 anos,0.216512,0.204462,0.193083,0.182338,0.172190,0.162607,0.153558,0.145012,...,0.115327,0.108909,0.102847,0.097124,0.091719,0.086614,0.081794,0.077242,0.072943,0.068884
2,Feminino,5 a 9 anos,0.111478,0.106165,0.101104,0.096285,0.091696,0.087325,0.083163,0.079199,...,0.065145,0.062040,0.059083,0.056267,0.053585,0.051031,0.048598,0.046282,0.044076,0.041975
3,Feminino,10 a 14 anos,0.161413,0.156809,0.152336,0.147990,0.143769,0.139668,0.135683,0.131813,...,0.117404,0.114055,0.110801,0.107641,0.104570,0.101587,0.098689,0.095874,0.093139,0.090482
4,Feminino,15 a 19 anos,0.398157,0.391452,0.384860,0.378379,0.372007,0.365743,0.359584,0.353528,...,0.330310,0.324747,0.319278,0.313902,0.308616,0.303419,0.298309,0.293286,0.288347,0.283491
5,Feminino,20 a 29 anos,0.533034,0.524953,0.516995,0.509157,0.501438,0.493837,0.486350,0.478977,...,0.450586,0.443756,0.437028,0.430403,0.423878,0.417452,0.411124,0.404891,0.398753,0.392708
6,Feminino,30 a 39 anos,0.994437,0.983453,0.972590,0.961847,0.951223,0.940717,0.930326,0.920050,...,0.880069,0.870348,0.860735,0.851227,0.841825,0.832527,0.823331,0.814237,0.805243,0.796349
7,Feminino,40 a 49 anos,2.050442,2.017548,1.985183,1.953336,1.922000,1.891167,1.860829,1.830977,...,1.716283,1.688750,1.661658,1.635002,1.608773,1.582965,1.557570,1.532583,1.507998,1.483806
8,Feminino,50 a 59 anos,4.623589,4.532837,4.443867,4.356642,4.271130,4.187296,4.105108,4.024533,...,3.717740,3.644768,3.573228,3.503093,3.434334,3.366925,3.300839,3.236050,3.172533,3.110262
9,Feminino,60 a 69 anos,10.405856,10.152650,9.905605,9.664572,9.429404,9.199959,8.976096,8.757681,...,7.935889,7.742785,7.554379,7.370559,7.191211,7.016227,6.845501,6.678930,6.516411,6.357847


In [102]:
categorias = list(df_mortalidade.columns[:3])

In [97]:
dados_ponderados = (
    ((df_mortalidade.iloc[:,2:].loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['Menor 1 ano']))] * 1).to_numpy() +
    (df_mortalidade.iloc[:,2:].loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['1 a 4 anos']))] * 4).to_numpy() +
    (df_mortalidade.iloc[:,2:].loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['5 a 9 anos']))] * 5).to_numpy()) / 10
)

In [100]:
df_mortalidade.columns[2:]

Index(['2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030',
       '2031', '2032', '2033', '2034', '2035', '2036', '2037', '2038', '2039',
       '2040', '2041', '2042'],
      dtype='object')

In [101]:

pd.DataFrame(data = dados_ponderados, columns=df_mortalidade.columns[2:], index=['0 a 9 anos'])

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,...,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042
0 a 9 anos,0.813048,0.778553,0.745541,0.713948,0.683711,0.654772,0.627074,0.600562,0.575186,0.550895,...,0.505384,0.484076,0.463678,0.444149,0.425452,0.407552,0.390413,0.374004,0.358292,0.343248


In [ ]:

df_temp = pd.DataFrame(data: {'Faixa Etária': [faixa_etaria],
                            'Constante': [constante], 'Coeficiente': [coeficiente]})

pd.DataFrame(



)

df_temp = pd.DataFrame({'Faixa Etária': [faixa_etaria],
                            'Constante': [constante], 'Coeficiente': [coeficiente]})

In [96]:
(
    ((df_mortalidade.iloc[:,2:].loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['Menor 1 ano']))] * 1).to_numpy() +
    (df_mortalidade.iloc[:,2:].loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['1 a 4 anos']))] * 4).to_numpy() +
    (df_mortalidade.iloc[:,2:].loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['5 a 9 anos']))] * 5).to_numpy()) / 10
)

array([[0.81304754, 0.77855272, 0.74554115, 0.71394804, 0.68371145,
        0.65477222, 0.6270738 , 0.60056216, 0.57518567, 0.55089498,
        0.52764292, 0.50538442, 0.48407638, 0.46367762, 0.44414876,
        0.42545218, 0.40755189, 0.39041349, 0.37400409, 0.35829225,
        0.34324791]])

In [92]:
df_mortalidade.iloc[:,2:].loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['Menor 1 ano']))]

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,...,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042
0,6.707037,6.436855,6.177556,5.928703,5.689874,5.460667,5.240692,5.029579,4.82697,4.632523,...,4.266813,4.094931,3.929973,3.77166,3.619725,3.47391,3.333969,3.199665,3.070771,2.94707


In [74]:
df_mortalidade.loc[(df_mortalidade.reset_index()['Sexo'] == 'Feminino').values & (df_mortalidade.reset_index()['Faixa Etária'].isin(['1 a 4 anos']).values)]

,,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,...,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042
Sexo,Faixa Etária,,,,,,,,,,,,,,,,,,,,,
Feminino,1 a 4 anos,0.216512,0.204462,0.193083,0.182338,0.17219,0.162607,0.153558,0.145012,0.136942,0.12932,...,0.115327,0.108909,0.102847,0.097124,0.091719,0.086614,0.081794,0.077242,0.072943,0.068884


In [52]:
df_mortalidade.loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['5 a 9 anos']))]

,Sexo,Faixa Etária,2022,2023,2024,2025,2026,2027,2028,2029,...,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042
2,Feminino,5 a 9 anos,0.111478,0.106165,0.101104,0.096285,0.091696,0.087325,0.083163,0.079199,...,0.065145,0.06204,0.059083,0.056267,0.053585,0.051031,0.048598,0.046282,0.044076,0.041975


In [ ]:
df_mortalidade.loc[(df_mortalidade['Sexo'] == 'Feminino') & (df_mortalidade['Faixa Etária'].isin(['Menor 1 ano']))]

In [35]:
map_faixas_etarias = {
    'Menor 1 ano':'0 a 9 anos'
    '1 a 4 anos':
    '5 a 9 anos':
    '10 a 14 anos':
    '15 a 19 anos': 
}

[]

array(['Menor 1 ano', '1 a 4 anos', '5 a 9 anos', '10 a 14 anos',
       '15 a 19 anos', '20 a 29 anos', '30 a 39 anos', '40 a 49 anos',
       '50 a 59 anos', '60 a 69 anos', '70 a 79 anos', '80 anos e mais'],
      dtype=object)

In [33]:
'20 a 29 anos', '30 a 39 anos', '40 a 49 anos',
       '50 a 59 anos', '60 a 69 anos', '70 a 79 anos', '80 anos e mais'
df_populacao.loc[df_populacao['Sexo'] == 'Feminino' & df_populacao['Faixa Etária'].isin(['Menor 1 ano', '1 a 4 anos']) == 'Feminino']


,Sexo,Faixa Etária,2022
0,Feminino,Menor 1 ano,22722
1,Feminino,1 a 4 anos,96868
2,Feminino,5 a 9 anos,117868
3,Feminino,10 a 14 anos,114922
4,Feminino,15 a 19 anos,123448
5,Feminino,20 a 29 anos,281189
6,Feminino,30 a 39 anos,280637
7,Feminino,40 a 49 anos,267790
8,Feminino,50 a 59 anos,223819
9,Feminino,60 a 69 anos,166719


In [21]:
df_natalidade

,Faixa Etária,Constante,Coeficiente,2022,2023,2024,2025,2026,2027,2028,...,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042
0,10 a 14 anos,3.613036,0.031106,1.560003,1.512225,1.465910,1.421013,1.377492,1.335303,1.294407,...,1.107962,1.074029,1.041134,1.009247,0.978337,0.948373,0.919327,0.891171,0.863877,0.837419
1,15 a 19 anos,94.016262,0.032437,39.160499,37.910631,36.700653,35.529294,34.395321,33.297541,32.234797,...,27.408728,26.533935,25.687063,24.867220,24.073544,23.305199,22.561377,21.841295,21.144196,20.469346
2,20 a 29 anos,118.450589,0.019836,69.333486,67.971748,66.636755,65.327981,64.044913,62.787044,61.553881,...,55.741986,54.647191,53.573897,52.521684,51.490136,50.478849,49.487424,48.515470,47.562606,46.628457
3,30 a 39 anos,51.444790,-0.006576,61.440660,61.846049,62.254114,62.664870,63.078337,63.494532,63.913473,...,66.050008,66.485811,66.924488,67.366061,67.810546,68.257965,68.708335,69.161677,69.618011,70.077355
4,40 a 49 anos,5.181765,-0.006901,6.243105,6.286339,6.329872,6.373707,6.417845,6.462289,6.507041,...,6.735491,6.782135,6.829102,6.876394,6.924013,6.971962,7.020243,7.068859,7.117811,7.167102
5,50 a 59 anos,0.038731,0.047289,0.010803,0.010304,0.009828,0.009374,0.008941,0.008528,0.008134,...,0.006422,0.006125,0.005842,0.005572,0.005315,0.005069,0.004835,0.004612,0.004399,0.004196


In [20]:
df_saldo_migratorio

,Sexo,"Idade calculada, anos, classe",Total
0,Feminino,5 a 9 anos,19.0
1,Feminino,10 a 14 anos,-40.8
2,Feminino,15 a 19 anos,498.0
3,Feminino,20 a 29 anos,414.0
4,Feminino,30 a 39 anos,-526.4
5,Feminino,40 a 49 anos,-152.4
6,Feminino,50 a 59 anos,-359.0
7,Feminino,60 a 69 anos,-129.0
8,Feminino,70 a 79 anos,37.4
9,Feminino,80 anos e mais,38.6


In [19]:
df_saldo_migratorio

,Sexo,"Idade calculada, anos, classe",Total
0,Feminino,5 a 9 anos,19.0
1,Feminino,10 a 14 anos,-40.8
2,Feminino,15 a 19 anos,498.0
3,Feminino,20 a 29 anos,414.0
4,Feminino,30 a 39 anos,-526.4
5,Feminino,40 a 49 anos,-152.4
6,Feminino,50 a 59 anos,-359.0
7,Feminino,60 a 69 anos,-129.0
8,Feminino,70 a 79 anos,37.4
9,Feminino,80 anos e mais,38.6


In [2]:
cod_municipios_rmc= [4100202,4100301,4100400,4101804,4102307,4103107,4104006,4104105,
                     4104204,4104253,4105201,4105805,4106209,4106902,4107652,4111258,
                     4113205,4114302,4119103,4119152,4119509,4120804,4121208,4122206,4122305,
                     4125506,4127601,4127882,4128633]

In [3]:
colunas_a_importar = [
    'Código do Item Geográfico', 'Item Geográfico',
    'Item Temporal',
    'Residência em 31/07/2005, UF', 'Residência em 31/07/2005, UF (código)',
    'Residência em 31/07/2005, UF (ordem)',
    'Frequência', 'Observações']

df_rec_migratorio_ufs = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - moradores municipios mesorregiao curitiba de acordo com UF pais de moradia em 2005.csv'
    , sep=';'
    , na_values='X'
    , usecols=colunas_a_importar

)

df_rec_migratorio_ufs = df_rec_migratorio_ufs[df_rec_migratorio_ufs['Código do Item Geográfico'].isin(cod_municipios_rmc)]

df_rec_migratorio_ufs.head(2)

,Código do Item Geográfico,Item Geográfico,Item Temporal,"Residência em 31/07/2005, UF","Residência em 31/07/2005, UF (código)","Residência em 31/07/2005, UF (ordem)",Frequência,Observações
27,4100202,ADRIANÓPOLIS,31/07/2010,"ES, Espirito Santo",32,8,NaN,NaN
28,4100202,ADRIANÓPOLIS,31/07/2010,"GO, Goias",52,9,NaN,NaN


In [4]:
df_rec_migratorio_ufs['Frequência'].sum()

107092.0

### Carregamento dos Dados de residentes da RMC que moravam em outros países em 2005

In [5]:
colunas_a_importar = [
    'Código do Item Geográfico', 'Item Geográfico',
    'Item Temporal', 'Residência em 31/07/2005, UF, município ou país estrangeiro',
    'Frequência', 'Observações']

df_rec_migratorio_extrangeiros = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - moradores municipios mesorregiao curitiba moravam em outro país em 2005.csv'
    , sep=';'
    , na_values='X'
    , usecols=colunas_a_importar

)

df_rec_migratorio_extrangeiros = df_rec_migratorio_extrangeiros[df_rec_migratorio_extrangeiros['Código do Item Geográfico'].isin(cod_municipios_rmc)]

df_rec_migratorio_extrangeiros.head(2)

,Código do Item Geográfico,Item Geográfico,Item Temporal,"Residência em 31/07/2005, UF, município ou país estrangeiro",Frequência,Observações
1,4100400,ALMIRANTE TAMANDARÉ,31/07/2010,País estrangeiro,73.0,9.0
3,4101804,ARAUCÁRIA,31/07/2010,País estrangeiro,84.0,7.0


In [6]:
df_rec_migratorio_extrangeiros['Frequência'].sum()

8252.0

### Carregamento dos Dados de residentes da RMC que moravam em outros municípios do PR em 2005

In [7]:
colunas_a_importar = [
    'Código do Item Geográfico', 'Item Geográfico',
    'Item Temporal', 'Residência em 31/07/2005, município','Residência em 31/07/2005, município (código)',
    'Frequência', 'Observações']

df_recebimento_migratorio_PR = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - moradores municipios mesorregiao curitiba de acordo com moradia em outro municipio PR em 2005.csv'
    , sep=';'
    , na_values='X'
    , usecols=colunas_a_importar

)

df_recebimento_migratorio_PR = df_recebimento_migratorio_PR[df_recebimento_migratorio_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]

df_recebimento_migratorio_PR.head(2)

,Código do Item Geográfico,Item Geográfico,Item Temporal,"Residência em 31/07/2005, município","Residência em 31/07/2005, município (código)",Frequência,Observações
328,4100202,ADRIANÓPOLIS,31/07/2010,ADRIANÓPOLIS,4100202,221.0,82.0
329,4100202,ADRIANÓPOLIS,31/07/2010,ALMIRANTE TAMANDARÉ,4100400,NaN,NaN


In [8]:
df_recebimento_migratorio_PR = df_recebimento_migratorio_PR[~df_recebimento_migratorio_PR['Residência em 31/07/2005, município (código)'].isin(cod_municipios_rmc)]

In [9]:
df_recebimento_migratorio_PR['Frequência'].sum()

54289.0

### Pessoas na RMC que moravam em outros estados, países ou municípios do Paraná exclusive aqueles da RMC

In [10]:
imigrantes_quinquenio = (df_recebimento_migratorio_PR['Frequência'].sum() + df_rec_migratorio_extrangeiros['Frequência'].sum() + df_rec_migratorio_ufs['Frequência'].sum())

print(f'Imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio)}')
print(f'Média Anual de imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio / 5)}')

Imigrantes entre 2005 e 2010: 169633
Média Anual de imigrantes entre 2005 e 2010: 33926


## Pessoas que saíram da RMC entre 2005 e 2010

In [11]:
df_perda_migratoria_PR = pd.read_csv(
    '../source/tab/originais/bme - migracao 2010 - pessoas que sairam dos 29 municípios da RMC desde 2005.csv'
    , sep=';'
    , na_values='X'
#    , usecols=colunas_a_importar

)


territorios_a_ignorar = ['ANTONINA', 'Metropolitana de Curitiba', 'GUARAQUEÇABA', 'GUARATUBA', 'MATINHOS', 'MORRETES', 'PARANAGUÁ', 'PORTO AMAZONAS']

df_perda_migratoria_PR = df_perda_migratoria_PR[~df_perda_migratoria_PR['Item Geográfico'].isin(territorios_a_ignorar)]

df_perda_migratoria_PR.head(2)

,Código do Nível Geográfico,Nível Geográfico,Código do Item Geográfico,Item Geográfico,Ordem Geográfica,Código do Item Temporal,Nível Temporal,Item Temporal,"Residência em 31/07/2005, município","Residência em 31/07/2005, município (código)","Residência em 31/07/2005, município (ordem)",Frequência,Observações
0,9000,País,76,Brasil,1,2010,8030,31/07/2010,ADRIANÓPOLIS,4100202,4100202,849.0,188.0
1,9000,País,76,Brasil,1,2010,8030,31/07/2010,AGUDOS DO SUL,4100301,4100301,900.0,120.0


### Pessoas que mudaram entre municípios da RMC, entre 2005 e 2010

In [12]:
df_perda_migratoria_PR[df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum()

177568.0

### Pessoas que saíram de Municípios da RMC (incluindo outros municípios)

In [13]:
df_perda_migratoria_PR[~df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum()

308811.0

In [14]:
Pessoas_sairam_curitiba = df_perda_migratoria_PR[~df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum() - df_perda_migratoria_PR[df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)]['Frequência'].sum()

In [15]:
print(f'Imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio)}')
print(f'Média Anual de imigrantes entre 2005 e 2010: {int(imigrantes_quinquenio / 5)}')
print(f'Emigrantes entre 2005 e 2010: {int(Pessoas_sairam_curitiba)}')
print(f'Média Anual de Emigrantes entre 2005 e 2010: {int(Pessoas_sairam_curitiba / 5)}')
print(f'Saldo Migratório entre 2005 e 2010: {int(imigrantes_quinquenio - Pessoas_sairam_curitiba)}')
print(f'Saldo Migratório Médio Anual entre 2005 e 2010: {int((imigrantes_quinquenio - Pessoas_sairam_curitiba) / 5)}')


Imigrantes entre 2005 e 2010: 169633
Média Anual de imigrantes entre 2005 e 2010: 33926
Emigrantes entre 2005 e 2010: 131243
Média Anual de Emigrantes entre 2005 e 2010: 26248
Saldo Migratório entre 2005 e 2010: 38390
Saldo Migratório Médio Anual entre 2005 e 2010: 7678


In [16]:
df_perda_migratoria_PR[~df_perda_migratoria_PR['Código do Item Geográfico'].isin(cod_municipios_rmc)].head(60)

,Código do Nível Geográfico,Nível Geográfico,Código do Item Geográfico,Item Geográfico,Ordem Geográfica,Código do Item Temporal,Nível Temporal,Item Temporal,"Residência em 31/07/2005, município","Residência em 31/07/2005, município (código)","Residência em 31/07/2005, município (ordem)",Frequência,Observações
0,9000,País,76,Brasil,1,2010,8030,31/07/2010,ADRIANÓPOLIS,4100202,4100202,849.0,188.0
1,9000,País,76,Brasil,1,2010,8030,31/07/2010,AGUDOS DO SUL,4100301,4100301,900.0,120.0
2,9000,País,76,Brasil,1,2010,8030,31/07/2010,ALMIRANTE TAMANDARÉ,4100400,4100400,5852.0,588.0
3,9000,País,76,Brasil,1,2010,8030,31/07/2010,ARAUCÁRIA,4101804,4101804,7565.0,838.0
4,9000,País,76,Brasil,1,2010,8030,31/07/2010,BALSA NOVA,4102307,4102307,682.0,82.0
5,9000,País,76,Brasil,1,2010,8030,31/07/2010,BOCAIÚVA DO SUL,4103107,4103107,796.0,128.0
6,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPINA GRANDE DO SUL,4104006,4104006,3198.0,405.0
7,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPO DO TENENTE,4104105,4104105,695.0,116.0
8,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPO LARGO,4104204,4104204,7132.0,833.0
9,9000,País,76,Brasil,1,2010,8030,31/07/2010,CAMPO MAGRO,4104253,4104253,1758.0,175.0


### Emigração Internacional 2005 a 2010

In [52]:
df_perda_migratoria_internacional = pd.read_csv(
    '../source/tab/originais/bme migracao 2010 - pessoas que sairam dos 29 municípios da RMC para outro pais 2005 a 2010 sem faixas etárias.csv'
    , sep=';'
    ,na_values='X'
)

df_perda_migratoria_internacional[
    (~df_perda_migratoria_internacional['Código do Item Geográfico'].isin(cod_municipios_rmc)) 
    & 
    (df_perda_migratoria_internacional['Item Geográfico'] != 'Metropolitana de Curitiba')
    
    ].groupby(by='Emigrante, sexo')['Frequência'].sum()


# Cálculo Emigrantes anuais da RMC entre 2005 e 2010, subtraindo os dados da Mesorregião Metropolitana
((df_perda_migratoria_internacional[
    (df_perda_migratoria_internacional['Item Geográfico'] == 'Metropolitana de Curitiba')
    ].groupby(by='Emigrante, sexo')['Frequência'].sum() ## emigrantes, agrupados por sexo e de toda Mesorregião
    -
df_perda_migratoria_internacional[
    (~df_perda_migratoria_internacional['Código do Item Geográfico'].isin(cod_municipios_rmc)) 
    & 
    (df_perda_migratoria_internacional['Item Geográfico'] != 'Metropolitana de Curitiba')## emigrantes, agrupados por sexo do litoral
    
    ].groupby(by='Emigrante, sexo')['Frequência'].sum()) /  5.5 #agrupamento por sexo e divisão por 5,5 anos
).astype(int)

Emigrante, sexo
Feminino     947
Masculino    896
Name: Frequência, dtype: int64